In [4]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
#first line push

# Posts and Users

In [5]:
composer = pd.DataFrame({'user_id':[1,2,1,3,4,1], 
                         'date':['2020-01-01', '2020-01-01', '2020-01-01', '2020-01-02', '2020-01-03', '2020-01-01'], 
                         'event':['enter', 'post', 'post', 'enter', 'enter', 'exit']})
user = pd.DataFrame({'user_id':[1,2,3,4,5], 
                     'country':['us', 'uk', 'us', 'us', 'chn'], 
                     'date':['2020-01-01', '2020-01-02', '2020-01-01', '2020-01-01', '2020-01-03'], 
                     'dau_flag':[1,1,1,0,1]})

### Q1 what is the post success rate for each day in the last week?

In [6]:
df = composer.copy()
df['date'] = pd.to_datetime(df.date)
# df = df[(datetime.datetime.now() - df.date).dt.days <= 7 ]
df['enter'] = np.where(df.event == 'enter', 1, 0)
df['post'] = np.where(df.event == 'post', 1, 0)
np.round(df.groupby('date')['post'].sum()/df.groupby('date')['enter'].sum(),2)

date
2020-01-01    2.0
2020-01-02    0.0
2020-01-03    0.0
dtype: float64

In [ ]:
select date, sum(case when event = 'post' then 1 else 0 end)/sum(case when event = 'enter' then 1 else 0 end)
from composer
where date_diff('day', date, current_date) <= 7
group by date

In [ ]:
select date, sum(case when event = 'post' then 1 else 0 end)/sum(case when event = 'enter' then 1 else 0 end) from composer
where date_diff('day', date, current_date) <= 7
group by date

### Q2 what is the average number of post per daily active user by country today? 

In [46]:
df = composer.copy()
df['date'] = pd.to_datetime(df.date)
# df = df[(datetime.datetime.now() - df.date).dt.days == 0]
df = df.merge(user[user.dau_flag == 1], on = 'user_id', how = 'left')
temp = df[df.event == 'post'].groupby('country')['user_id'].count()
print temp
print temp.mean()

country
uk    1
us    1
Name: user_id, dtype: int64
1.0


In [ ]:
select country, avg(cnt)
from
(select country, user_id, count(*) as cnt
from composer
left join user
on composer.user_id = user.user_id
where date_diff('day', date, current_date) = 0 and dau_flag = 1 and event = 'post'
group by country, user_id) temp
group by country

In [ ]:
Select country, sum(case when event = ‘post’then 1 else 0 end)/count(distinct user_id) as post_per_user
from
(Select * from composer
Left join user
On user_id = user_id
Where dau_flag = 1 and date = current_date) as temp
Group by country


In [ ]:
select country, count(*)/count(distinct user_id) from composer
left join user
on composer.user_id = user.user_id
where date_diff('day', composer.date, current_date) = 0 and dau_flag = 1 and event = 'post'
group by country

# Session

In [43]:
table1 = pd.DataFrame({'date': ['2020-01-01', '2020-01-01', '2020-01-01', '2020-01-02', '2020-01-03', '2020-01-01'],
                       'session_id':[1,2,3,4,5,6],
                       'user_id':[1,2,1,1,3,4],
                       'act':['enter', 'enter', 'post', 'exit', 'enter', 'post']
                      })
table2 = pd.DataFrame({'date':['2020-01-01', '2020-01-01', '2020-01-01', '2020-01-02', '2020-01-03'],
                       'session_id':[1,2,3,4,5],
                       'time_spent':[20,40,50,10,100]
                      })


### Q1：generate average number of sessions per user per day

In [48]:
df = table1.copy()
temp = df.groupby(['date', 'user_id'])['session_id'].nunique()
print temp 
print temp.mean()

date        user_id
2020-01-01  1          2
            2          1
            4          1
2020-01-02  1          1
2020-01-03  3          1
Name: session_id, dtype: int64
1.2


In [ ]:
select date, avg(cnt)
from
(select date, user_id, count(distinct session_id) as cnt from table1
group by date, user_id) temp
group by date

In [ ]:

select avg(cnt)
from (select date, count(distinct session_id)/count(distinct user_id) as cnt from table1
group by date)

In [ ]:
select date, avg(count(distinct session_id)/count(distinct user-id))
from table1
group by date

### Q2: Generate number of users per time interval in order to measure how many user is spending certain amount of time

In [51]:
df = table2.copy()
df['interval'] = df.time_spent/10.0+1
df.merge(table1, on = ['session_id', 'date'], how = 'left').groupby('interval')['user_id'].nunique().reset_index()

,interval,user_id
0,2.0,1
1,3.0,1
2,5.0,1
3,6.0,1
4,11.0,1


In [ ]:
select time_spent/10.0+1 as interval, count(user) as cnt
from table2
left join table1
on table2.session_id = table1.session_id
group by interval

In [ ]:
select (time_spend/10.0)+1 as interval, count(user) from table2
left join table1
on table2.session_id = table1.session_id
group by interval

# Country User

In [60]:
table1 = pd.DataFrame({'country':['us', 'us', 'uk', 'chn', 'chn', 'us'],
                      'userid':[1,2,3,4,5,6]
                      })
table2 = pd.DataFrame({'userid':[1,1,2,2,3,4,4,5],
                      'post_date':['2020-01-01', '2020-01-01', '2020-01-01', '2020-01-02', '2020-01-03',
                                  '2020-01-01', '2020-01-01', '2020-01-01']
                      })

### Q1 number of users in each country

In [54]:
df = table1.copy()
df.groupby('country')['userid'].nunique()

country
chn    2
uk     1
us     3
Name: userid, dtype: int64

In [ ]:
select country, count(distinct userid) from table1
group by country

### Q2 user active rate (at least one post) yesterday per country

In [66]:
df = table2.copy()
# df = df[datetime.datetime.now() - df.post_date == 1]
df = df.merge(table1, how = 'right', on = 'userid')
df['active'] = np.where(df.post_date.isnull(), 0, 1)
df.groupby('country')['active'].sum()/(df.groupby('country')['userid'].count()+0.0)

country
chn    1.0
uk     1.0
us     0.8
dtype: float64

In [ ]:
select country, sum(case when post_date is null then 0 else 1)/count(*) from table2
right join table1
on table1.userid = table2.userid
where date_diff('day', post_dat, current_date) = 1
group by country

In [ ]:
select country, sum(case when post_date is null then 0 else 1 end)/count(distinct userid) from table1
left join table2
on table2.userid = table1.userid
where date_diff('day', post_date, current_date) = 1
group by country

# SPAM

In [77]:
table = pd.DataFrame({'date':['2020-01-01', '2020-01-01', '2020-01-01', '2020-01-02', '2020-01-03',
                                  '2020-01-01', '2020-01-01', '2020-01-01'],
                     'user_id':[1,1,2,2,3,4,4,5],
                     'post_id':[1,2,3,4,5,6,7,8],
                     'content_type':['view', 'comment', 'photo', 'report',
                                     'view', 'comment', 'photo', 'report'],
                     'extra':['text', 'SPAM', 'la', 'SPAM', 
                              'SPAM', 'text', 'SPAM', 'la']
                     })
review_removal = pd.DataFrame({'date':['2020-01-01', '2020-01-01', '2020-01-01', '2020-01-02', '2020-01-03'],
                              'reviewer_id':[1,2,1,1,3],
                              'post_id':[1,2,4,6,3]})



### Q1 What is the total number of posts posted yesterday for each type of report?

In [74]:
df = table.copy()
# df = df[datetime.datetime.now() - df.date == 1]
df[df.content_type == 'report'].groupby('extra')['post_id'].count().reset_index()

,extra,post_id
0,SPAM,1
1,la,1


In [ ]:
select extra, count(distinct post_id) as cnt
from table
where date_diff('day', date, current_date) = 1 and content_type = 'report'
group by extra

In [ ]:
select extra, count(distinct post_id) from table
where date_diff('day', date, current_date) = 1 and content_type = 'report'
group by extra

### Q2 What percentage of the posts viewed by the users yesterday is reported as spam?

In [79]:
df = table.copy()
# df = df[datetime.datetime.now() - df.date == 1]
df = df[df.content_type == 'view']
df = df.merge(review_removal, on = 'post_id', how = 'left')
df['spam'] = np.where(df.reviewer_id.isnull(),0,1)
df['spam'].sum()/(df['spam'].count()+0.0)

0.5

In [ ]:
select sum(case when reviewer_id is null then 0 else 1 end)/count(distinct post_id) as ratio
from table
left join review_removal
on post_id = post_id
where date_diff('day', table.date, current_date) = 1 and content_type = 'view'

In [ ]:
select sum(case when reviewer_id is null then 0 else then 1 end)/count(distinct post_id) from table
left join reviewer_removal
on table.post_id = reviewer_removal.post_id
where date_diff('day', table.date, current_date) = 1  and content_type = 'view'

# Survey question skip

##### 刚加入的用户会要求填一份调查问卷，但问卷里的问题也可以跳过，每道题只要被用户见到就会生成一条记录（event为saw），如果被回答或者被跳过会生成另一条数据（即每道题每个用户都会有两条记录），回答则event为answered，跳过即为skip。并且每道题出现在每个用户前的顺序有可能不同，所以有question_order。假设该表里已经存了1M用户的数据，在一个新用户进来时，如何安排题目尽可能多地得到新用户的答案，减少skip？

In [117]:
survey_log = pd.DataFrame({'user_id':[1,1,2,2,2,2,3,3,4,4,4,4],
                          'question_id':[1,1,2,2,3,3,4,4,5,5,6,6],
                          'question_order':[1,1,3,3,2,2,4,4,6,6,5,5],
                          'event':['saw', 'answered', 'saw', 'answered', 'saw', 'answered', 
                                   'saw', 'skipped', 'saw', 'answered', 'saw', 'skipped'],
                          'timestamp':['08:00', '08:01', '08:00', '08:02', '08:00', '08:01',
                                      '08:01', '08:01', '08:02', '08:04', '08:00', '08:01']
                          })

### Q1 每道题的回答率，注意不能只求回答次数，要除以总的看见此题的次数

In [118]:
df = survey_log.copy()
df = (df[df.event == 'answered'].groupby('question_id')['user_id'].count()/
 (df[df.event == 'saw'].groupby('question_id')['user_id'].count()+0.0))
answer_rate = df.reset_index().sort_values(by = 'user_id', ascending = False)

In [119]:
answer_rate

,question_id,user_id
0,1,1.0
1,2,1.0
2,3,1.0
4,5,1.0
3,4,NaN
5,6,NaN


In [ ]:
select question_id, 
sum(case when event = 'answered' then 1 else 0 end)/sum(case when event = 'saw' then 1 else 0 end) as ratio
from survey_log
group by question_id
order by ratio desc

In [ ]:
select question_id,
sum(case when event = 'answered' then 1 else 0 end)/sum(case when event = 'saw' then 1 else 0 end) as ratio
from survey_log
group by question_id
order by ratio desc

### Q2 即使按照回答率对题目进行排序，如果新来的用户已经skip掉了回答率最高和次高的题，如何动态调整题目顺序，获得此用户尽可能多的回答

#### ANSWER - find the users who also skip those two questions and see what their third question is

In [132]:
first_users = survey_log[(survey_log.question_order==1) &
           (survey_log.question_id==answer_rate['question_id'].iloc[0]) &
           (survey_log.event == 'answered')]['user_id']
second_users = survey_log[(survey_log.question_order==2) &
           (survey_log.question_id==answer_rate['question_id'].iloc[1]) &
           (survey_log.event == 'answered')]['user_id']
historical_users = list(set(first_users).intersection(set(second_users)))
historical_users = [1,2,3,4,5,6]
third_question = list(survey_log[(survey_log.question_order == 3) &
           (survey_log.user_id.isin(historical_users)) &
          (survey_log.event == 'answered')]['question_id'])

((survey_log[(survey_log.event == 'answered')&(survey_log.question_id.isin(third_question))].groupby('question_id')['user_id'].count()/
 (survey_log[(survey_log.event == 'saw')&(survey_log.question_id.isin(third_question))].groupby('question_id')['user_id'].count()+0.0))
.reset_index().sort_values(by = 'user_id', ascending = False))

,question_id,user_id
0,2,1.0


In [ ]:
with top_two as (
select question_id
from survey_log
group by question_id
order by sum(case when event = 'answered' then 1 else 0 end)/sum(case when event = 'saw' then 1 else 0 end) desc
limit 2),

top_two_skip_user as
(select distinct user_id from survey_log
where question_id in top_two and event = 'skipped'
group by user_id
having count(distinct question_id)=2)

select question_id from survey_log
where user_id is in top_two_skip_user and question_order = 3 and event = 'answered'
group by question_id
order by count(distinct user_id) DESC
LIMIT 1

In [ ]:
with top_two as (
select question_id
from survey_log
group by question_id
order by sum(case when event = 'answered' then 1 else 0 end)/sum(case when event = 'saw' then 1 else 0 end) desc
    limit 2
),
top_two_skip_user as 
(
select user_id from survey_log
where question_id in top_two and event = 'skipped'
    group by user_id
    having count(question_id)=2
)

select question_id from survey_log
where user_id in top_two_skip_user and question_id not in top_two
group by question_id
order by sum(case when event = 'answered' then 1 else 0 end)/sum(case when event = 'saw' then 1 else 0 end) desc
limit 1

# Friend interactions

In [134]:
friend = pd.DataFrame({'user_id_1':[1,2,3,4,5,6,7,8],
                       'user_id_2':[2,8,5,5,6,1,6,6]})
interaction = pd.DataFrame({'sender_id':[1,1,2,3,4,4,5,6,7,7,6],
                            'receipient_id':[2,6,1,5,5,5,6,5,6,6,5],
                            'action':['comment', 'comment', 'share', 'share', 'share',
                                     'comment', 'comment', 'share', 'comment', 'share', 'comment']})


### Q1 问怎么知道每队好友互动(comment _ share)多少次呢

In [141]:
df = interaction.copy()
temp1 = df.merge(friend, left_on = ['sender_id', 'receipient_id'], right_on = ['user_id_1', 'user_id_2'], how = 'left')
temp2 = df.merge(friend, left_on = ['sender_id', 'receipient_id'], right_on = ['user_id_2', 'user_id_1'], how = 'left')
temp = pd.concat([temp1, temp2])
temp.groupby(['user_id_1', 'user_id_2', 'action']).count()

receipient_id  sender_id
user_id_1 user_id_2 action                           
1.0       2.0       comment              1          1
                    share                1          1
3.0       5.0       share                1          1
4.0       5.0       comment              1          1
                    share                1          1
5.0       6.0       comment              2          2
                    share                1          1
6.0       1.0       comment              1          1
7.0       6.0       comment              1          1
                    share                1          1

In [ ]:
--litez: may want to first clarify that no `duplicates` in friend, i.e., given (user_id_1:[1], user_id_2[2]), 
-- there is no (user_id_1:[2], user_id_2[1]), 

with temp1 as
(select * from interaction
 left join friend
 on sender_id = user_id_1 and receipient_id = user_id_2
),
temp2 as
(select * from interaction
 left join friend
 on sender_id = user_id_2 and receipient_id = user_id_1
)

select user_id_1, user_id_2, action, count(*) from
(select * from temp1
union
select * from temp2)
group by user_id_1, user_id_2, action

# Instagram adding video chat feature

In [145]:
table = pd.DataFrame({'date':['2018-05-05','2018-05-05','2018-05-06'],
                      'caller_id':[1234,1234,1234],
                      'receiver_id':[7567,3669,8998],
                      'duration':[63.4,50.6,0.0],
                      'caller_country':['ES', 'ES', 'ES'],
                      'receiver_country':['ES', 'ES', 'PT']
                     })


### Q: for people who used the feature *for the first time* on 2018-05-05, how many people used the feature X days later?

In [163]:
first_time = table.copy()
first_time = pd.concat([first_time.groupby('caller_id')['date'].min().reset_index().rename(columns = {'caller_id':'user_id'}),
 first_time.groupby('receiver_id')['date'].min().reset_index().rename(columns = {'receiver_id':'user_id'})])
first_time = first_time[first_time.date == '2018-05-05']
df = table[(table.caller_id.isin(first_time['user_id'])) | (table.receiver_id.isin(first_time['user_id']))]
df['date'] = pd.to_datetime(df.date)
df['date_diff'] = (df.date - pd.to_datetime('2018-05-05')).dt.days
df[df.date_diff>0].groupby('date_diff').count()

,caller_country,caller_id,date,duration,receiver_country,receiver_id
date_diff,,,,,,
1,1,1,1,1,1,1


In [ ]:
with caller_first as
(select caller_id from table
 group by caller_id
 having min(date) = '2018-05-05'
),
receiver_first as
(select receiver_id from table
 group by receiver_id
 having min(date) = '2018-05-05'
),
with first_user as
(select caller_id as user_id 
from caller_first
union
select * from receiver first
)

select 
    date_diff('day', '2018-05-05', date) as X, 
    --litez: a user_id can be both a caller and a receiver
    count(distinct user_id) 
from table
where caller_id in first_user or receiver_id in first_user and date != '2018-05-05'
group by X

In [ ]:
with first_time as
(select caller_id as user_id from table
 group by caller_id
 having min(date) = '2018-05-05'
 union
 select receiver_id as user_id from table
 group by receiver_id
 having min(date) = '2018-05-05'
)

select date_diff('day', date, '2018-05-05') as X, count(*) from table
where caller_id in first_time or receiver_id in first_time and date != '2018-05-05'
group by X

# Daily Active User

In [165]:
dau = pd.DataFrame({'user_id':[1,2,3,4,5,1,2,3,3,4],
                   'ds':['2018-05-05','2018-05-05','2018-05-06','2018-05-05','2018-05-05',
                        '2018-05-05','2018-05-07','2018-05-06', '2018-05-07','2018-05-05']})


### Q: Return a table which has columns: ds, user_ud, state. State includes 'new', 'churned', 'resurrected' and 'retained'


#### 新用户就是第一次来的用户。churned就是今天没有来的。resurrected就是昨天之前都没来的老用户今天来了。retained就是昨天来了今天也来了的老用户

In [178]:
dau['ds'] = pd.to_datetime(dau.ds)
today = dau[(pd.to_datetime('2018-05-07') - dau.ds).dt.days ==0]
yesterday = dau[(pd.to_datetime('2018-05-07') - dau.ds).dt.days ==1]
before_yesterday = dau[(pd.to_datetime('2018-05-07') - dau.ds).dt.days ==2]

df = today.merge(yesterday, on = 'user_id', how = 'outer', suffixes = ['_today', '_yesterday'])
df = df.merge(before_yesterday, on = 'user_id', how = 'outer').rename(columns = {'ds':'ds_before'})
df['state'] = np.where((df.ds_yesterday.isnull())&(df.ds_today.notnull()), 'new',
                       np.where((df.ds_yesterday.notnull())&(df.ds_today.notnull()), 'retained', 
                                np.where((df.ds_today.isnull()), 'churned', 
                      np.where((df.ds_yesterday.isnull())&(df.ds_before.isnull())&(df.ds_today.notnull()), 'resurrected', 'null'))))
print df

    ds_today  user_id ds_yesterday  ds_before     state
0 2018-05-07        2          NaT 2018-05-05       new
1 2018-05-07        3   2018-05-06        NaT  retained
2 2018-05-07        3   2018-05-06        NaT  retained
3        NaT        1          NaT 2018-05-05   churned
4        NaT        1          NaT 2018-05-05   churned
5        NaT        4          NaT 2018-05-05   churned
6        NaT        4          NaT 2018-05-05   churned
7        NaT        5          NaT 2018-05-05   churned


In [ ]:
with today as
(select * from dau
where date_diff('day', ds, current_date) = 0),
yesterday as
(select * from dau
where date_diff('day', ds, current_date) = 1),
before_yesterday as
(select * from dau
where date_diff('day', ds, current_date) = 2)

select case when yesterday.ds is null and today.ds not null then 'new'
        when today.ds is null then 'churned'
        when yesterday.ds not null and today.ds not null then 'retained'
        when before_yesterday.ds is null and yesterday.ds is null and today.ds not null then 'resurrected'
        else 'null' end as state
    
from today
full join yesterday
on today.user_id = yesterday.user_id 
full join before_yesterday
on yesterday.user_id = before_yesterday.user_id


# Follow friends



In [182]:
table1 = pd.DataFrame({'user_id':[1,2,3,4,5,6,7,8,9], 
                       'sports':['ft', 'ft', 'bs', 'bs', 'sw', 'sw', 'vb', 'vb', 'ft']}) 
table2 = pd.DataFrame({'follower_id':[1,2,3,4,5,6,7,8,9],
                       'followee_id':[2,5,6,1,1,3,4,5,6]})

### Q1 How many people are following each sports category?

In [183]:
df = table2.copy()
df.merge(table1, left_on = 'followee_id', right_on = 'user_id', how = 'left').groupby('sports')['follower_id'].nunique()

sports
bs    2
ft    3
sw    4
Name: follower_id, dtype: int64

In [ ]:
select sports, count(distinct follower_id) as cnt
from (select * from table2
left join table1
on followee_id = user_id)
group by sports

In [ ]:
select sports, count(distinct follower_id) as cnt from table2
left join table1
on table2.followee_id = table1.user_id
group by sports

### Q2 How many basketball players are following football players？

In [185]:
bsp = table2.copy()
bsp = bsp.merge(table1, left_on = 'follower_id', right_on = 'user_id', how = 'left')
bsp = bsp[bsp.sports == 'bs']

df = bsp.merge(table1, left_on = 'followee_id', right_on = 'user_id', how = 'left', suffixes = ('_follower', '_followee'))
df = df[df.sports_followee == 'ft']
df['followee_id'].nunique()

1

In [ ]:
select count(distinct follower_id) as cnt
from 
(select follower_id, followee_id, sports as sports_follower from table2
left join table1
on follower_id = user_id
where sports = 'basketball') as bsp
left join table1
on followee_id = user_id
where table1.sports = 'football'


Select basket as (Select user_name from sports
Where sports = ‘basketball’)
Select football as (select user_name from sports
Where sports = ‘football’)
Select count(distinct follower_id) from follow
Where follower_id is in basket and followee_id is in football


In [ ]:
select count(distinct follower_id) from table2
left join table1 a
on table2.follower_id = table1.user_id
left join table1 b
on table2.followee_id = table1.user_id
where a.sports = 'bs' and b.sports = 'ft'

# User Active or Banned

In [190]:
ad = pd.DataFrame({'id':[1,2,3,4,5,6,7,8,9,10],
                   'date':['2018-05-05','2018-05-05','2018-05-06','2018-05-05','2018-05-05',
                        '2018-05-05','2018-05-07','2018-05-06', '2018-05-07','2018-05-05'],
                  'amount':[123,456,789,135,235,678,346,256,7467,235],
                  'status':['banned', 'banned', 'active', 'active', 'banned',
                           'banned', 'active', 'active', 'banned', 'banned']})

### Q1 昨天百分之多少的正常advertiser是活跃的（有付钱）

In [193]:
df = ad.copy()
df['date'] = pd.to_datetime(df['date'])
# df = df[(datetime.datetime.now() - df['date']).dt.days == 1]
len(df[(df.status!='banned')&(df.amount>0)])/(len(df[df.status!='banned'])+0.0)

1.0

In [ ]:
select sum(case when amount > 0 then 1 else 0 end) / count(distinct id)
from ad
where date_diff('day', date, current_date) = 1 and status != 'banned'

In [ ]:
select sum(case when amount>0 then1 else 0 end)/count(distinct id) from ad
where date_diff('day', date, current_date) = 1 and status!= 'banned'

### Q2 昨天有多少被封了。注意是昨天被封的，不包括昨天以前被封的。

In [198]:
df = ad.copy()
df['date'] = pd.to_datetime(df.date)
df = df.loc[df[df.status == 'banned'].groupby('id')['date'].idxmin()]
len(df[(datetime.datetime.now() - df['date']).dt.days==10])

0

In [201]:
df.sort_values(by = ['id', 'date'], inplace=True)
df['pre'] = np.where(df['id'].shift(1) == df.id, df.status.shift(1), 'null')
(len(df[(df.status=='banned') &((df.pre!='banned')|(df.pre.isnull()))]))

6

In [ ]:
select *
from
(select *, lag(status, 1) over (partition by id order by date asc) as pre from ad) as temp
where status=='banned' and (pre!='banned' or pre is null)

In [ ]:
select count(*) 
from
(select *, lag(status, 1) over (partition by id order by date) as pre from ad
where status = 'banned' and (pre != 'banned' or pre is null)) and date_diff('day', date, current_date) = 1




with before_yesterday as (select id, date, status from ad
where date_diff('day', date, current_date) = 2)

select * from ad
left join before_yesterday
on ad.id = before_yesterday_id
where date_diff('day', ad.date, current_date) = 1 and ad.status = 'banned' 
and (before_yesterday.status != 'banned' or before_yesterday.status is null)


# Advertisement

In [219]:
table = pd.DataFrame({'Advertise_id':[1,1,2,3,3,3,4,5,6,6,5,2,7,7,4],
                      'User_id':[1,2,3,4,2,4,5,4,6,7,2,3,4,3,2],
                      'Time_stamp':['08:00', '08:01', '08:00', '08:02', '08:00', '08:01',
                                      '08:01', '08:01', '08:02', '08:04', '08:00', '08:01',
                                   '08:04', '08:06', '08:02'],
                      'Action':['view', 'view', 'click', 'click', 'click',
                               'view', 'click', 'view', 'hide', 'hide',
                                'click', 'view', 'hide', 'view', 'click']}) 

### Q1 找出过去十天表现最好的一个广告。注意，因为面试官没有直接告诉你怎么定义最好的广告，所以你要先和面试官确定是用click through rate 还是＃of clicks

In [212]:
df = table.copy()
# df = df[(datetime.datetime.now() - df.Time_stamp).dt.days <=10]
df = (df[df.Action == 'click'].groupby('Advertise_id')['User_id'].count()+0.0)/df[df.Action == 'view'].groupby('Advertise_id')['User_id'].count()
df.reset_index()['User_id'].max()

1.0

In [215]:
select Advertise_id, 
--litez: the corner case where denominator is zero
sum(case when Action = 'click' then 1 else 0 end)/sum(case when Action = 'view' then 1 else 0 end) as ctr
from table
where date_diff('day', Time_stamp, current_date) <=10
group by Advertise_id
order ctr desc
limit 1

SyntaxError: invalid syntax (<ipython-input-215-9aa9f47c9dc2>, line 1)

In [ ]:
select Advertise_id, 
sum(case when Action = 'click' then 1 else 0 end)/sum(case when Action = 'view' then 1 else 0 end) as ctr
from table
where date_diff('day', Time_stamp, current_date) <= 10
group by Advertise_id
order by ctr desc
limit 1

### Q2 如果我们给用户A看了广告X，用户A点击了广告X，请问我们要怎么决定接下来给客户A看哪个广告？这个其实有一点像SQL＋Case问题，因为首先你先要告诉面试官，要用怎么样的数据去推荐下一个广告

In [225]:
df = table.copy()
similar_user = df[(df.Advertise_id == 3) & ((df.Action == 'view')|(df.Action == 'click'))]
similar_user = similar_user.groupby(['User_id', 'Advertise_id'])['Action'].count().reset_index()
similar_user = similar_user[similar_user.Action==2]['User_id'].tolist()

next_ad = df[(df.User_id.isin(similar_user))&(df.Advertise_id != 3)]
next_ad = (next_ad[next_ad.Action == 'click'].groupby('Advertise_id')['User_id'].count()+0.0)/next_ad[next_ad.Action == 'view'].groupby('Advertise_id')['User_id'].count()
next_ad.reset_index().sort_values(by = 'User_id')

,Advertise_id,User_id
0,5,NaN


In [ ]:
with similar_user as (
select User_id from table
where Advertise_id = 'X' and  Action = 'click'

select Advertise_id,
sum(case when Action = 'click' then 1 else 0 end)/sum(case when Action = 'view' then 1 else 0 end) as ctr
from table
where User_id in similar_user and Advertise_id != 'X'
group by Advertise_id
order by ctr desc
limit 1

In [ ]:
with X_user as (select User_id from table
where Advertise_id = 'X' and Action = 'click')

select Advertise_id,
sum(case when Action = 'click' then 1 else 0 end)/sum(case when Action = 'view' then 1 else 0 end) as ctr
from table
where User_id in X_user and Advertise_id != 'X'
group by Advertise_id
order by ctr desc


# Video calls

In [234]:
video_calls = pd.DataFrame({'caller':[1,1,2,3,4,3,3,4,5,3],
                           'receipent':[2,58,3,8,9,2,2,2,4,1],
                           'ds':['2020-01-01', '2020-01-02', '2020-01-01', '2020-01-03', '2020-01-01',
                                '2020-01-01', '2020-01-02', '2020-01-01', '2020-01-03', '2020-01-01'], 
                           'call_id':[1,2,3,4,5,6,7,8,9,10],
                           'duration':[26,34,56,23,45,67,1,235,44,3]})
fb_dau = pd.DataFrame({'user_id':[1,2,3,4,5],
                      'DAU_flag':[1,0,1,1,1],
                      'ds':['2020-01-01', '2020-01-12', '2020-01-01', '2020-01-01', '2020-01-01'],
                      'country':['us', 'us', 'uk', 'uk', 'chn']})


### Q1 on '2020-01-01' how many people initiate multiple calls?

In [232]:
df = video_calls.copy()
df = df[df.ds == '2020-01-01']
df = df.groupby('caller')['call_id'].count().reset_index()
len(df[df.call_id>1])

2

In [ ]:
select count(caller) from
(select caller, count(distinct call_id) as cnt from video_calls
where ds = '2020-01-01'
group by caller
having cnt>1)

In [ ]:
select count(caller) from
(select caller, count(distinct call_id) as cnt
from video_calls
where ds = '2020-01-01'
group by caller
having cnt > 1)

### Q2 % of DAU used the video calls functions by each country on '2020-01-01'?

In [235]:
caller_table = video_calls.copy()
caller_table = caller_table[caller_table.ds == '2020-01-01']['caller']
receipent_table = video_calls.copy()
receipent_table = receipent_table[receipent_table.ds == '2020-01-01']['receipent']
df = fb_dau[(fb_dau.DAU_flag == 1)]
df['indicator'] = np.where(df.user_id.isin(list(set(caller_table).union(set(receipent_table)))), 1, 0)
(df.indicator.sum()+0.0)/len(df)

/Users/yinan.zheng/workplace/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0.75

In [ ]:
with user_table as
(select caller as user from video_calls
 where ds = '2020-01-01'
union
select recipent as user from video_calls
  where ds = '2020-01-01'
)

--litez: two corner cases not being captured here
--1) duplicates of user_id in both caller_table and receipient table????
--2) possibility of division by zero

select country, sum(case when(user_id in user_table) then 1 else 0 end) / count(distinct user_id) 
from fb_dau
where  DAU_flag = 1
group by country

In [ ]:
with users as (
select distinct user_id from
(select caller_id as user_id from video_calls
where ds = '2020-01-01'
union
select receiver_id as user_id from video_calls
where ds = '2020-01-01')
)

select country, sum(case when user_id in users then 1 else 0 end)/count(*) from fb_dau
where dau_flag = 1
group by country

# Friending

In [239]:
friending = pd.DataFrame({'sender_id':[1,1],
                         'receiver_id':[2,3],
                         'sent_date':['2019-09-15', '2019-10-15'],
                         'accepted_date':['2019-09-18', '2019-10-15'],
                         'sender_country':['US', 'US']})

fraud = pd.DataFrame({'userid':[1,2],
                     'fraud_type':['spam', 'phishing']})

### Q1 What is the same day acceptance rate for every day in the last week?

In [237]:
df = friending.copy()
# df = df[(datetime.datetime.now() - df.sent_date).dt.days <= 7]
df['flag'] = np.where(df.sent_date == df.accepted_date, 1, 0)
df.groupby('sent_date')['flag'].sum()/len(df)

sent_date
2019-09-15    0.0
2019-10-15    0.5
Name: flag, dtype: float64

In [ ]:
select sent_date, sum(case when sent_date = accepted_date then 1 else 0 end)/count(*)
from friending
where date_diff('day', sent_date, current_date) <=7 
group by sent_date

In [ ]:
select sent_date, sum(case when sent_date = accepted_date then 1 else 0 end)/count(*) from friending
where date_diff('day', sent_date, current_date) <=7
group by sent_date

### Q2 What is the percentage of fraud users send more than 10 friend requests yesterday by fraud type?

In [240]:
df = friending.copy()
# df = df[(datetime.datetime.now() - df.sent_date).dt.days == 1]
df = df.groupby('sender_id')['sent_date'].count().reset_index()
# df = df[df.sent_date>10]
temp = fraud.merge(df, left_on = 'userid', right_on = 'sender_id', how = 'left')
temp['flag'] = np.where(temp.sender_id.isnull(), 0, 1)
temp.groupby('fraud_type')['flag'].sum()/len(temp)

fraud_type
phishing    0.0
spam        0.5
Name: flag, dtype: float64

In [ ]:
select fraud_type, sum(case when sender_id is null then 0 else 1 end)/count(*)
from
fraud 
left join
(select sender_id, count(*) as cnt from friending
where date_diff('day', sent_date, current_date) = 1
group by sender_id
having cnt>10) as temp
on fraud.userid = temp.sender_id
group by fraud_type

In [ ]:
with temp as
(select sender_id as user_id
from friending
where date_diff('day', sent_date, current_date) = 1
group by sender_id
having count(*) >10)

select fraud_type, sum(case when userid in temp then 1 else 0 end)/count(*) from fraud
group by fraud_type

# Search Quality

In [9]:
table = pd.DataFrame({'search_id':[1,2,3,4,5,6,7,8,9,10],
                      'query':['dog', 'dog', 'dog','dog', 'dog', 'dog','cat', 'cat', 'cat', 'cat'],
                      'position':[1,3,2,4,5,6,3,1,2,4],
                      'result_relevance':[5,4,2,4,1,3,2,2,1,3]  
})

#### Position表示search 结果出现在页面的位置，相当于rank，正整数，越小代表越靠前。result_relevance代表search 结果跟query的相关程度，范围1到5， 越大代表越相关

### Q1 定义一个metric to measure search quality，既要考虑position 也要考虑相关程度，计算每个query的metric

In [17]:
df = table.copy()
# position_max = df.groupby('query')['position'].max().reset_index().rename(columns = {'position': 'position_max'})
# df = df.merge(position_max, how = 'left', on = 'query')
df['search_quality'] = df.result_relevance /(df.position)
raw = df.groupby('query')['search_quality'].sum().reset_index()
df['perfect_position'] = df.groupby('query')['result_relevance'].rank(method = 'dense', ascending = False)
df['perfect_quality'] = df.result_relevance /(df.perfect_position)
perfect = df.groupby('query')['perfect_quality'].sum().reset_index()
final = raw.merge(perfect, on = 'query', how = 'left')
final['final_quality'] = final.search_quality/final.perfect_quality
final

,query,search_quality,perfect_quality,final_quality
0,cat,3.916667,5.333333,0.734375
1,dog,9.033333,10.700000,0.844237


In [249]:
with position_max as
(
select query, max(position) as position_max from table
group by query)

select table.*, result_relevance/(position/position_max) as search_quality
from table
left join position_max
on query
order by query, search quality Desc


SyntaxError: invalid syntax (<ipython-input-249-cfaab06962e2>, line 1)

In [ ]:
--litez: compute NDCG as the metric
https://en.wikipedia.org/wiki/Discounted_cumulative_gain

In [ ]:
with raw as (select query, sum(result_relevance/position) as search_quality from table
group by query
order by query, search_quality desc),

perfect as
select query, sum(result_relevance/perfect_position) as perfect_quality
from
(select *,
 dense_rank() over(partition by query order by result_relevance desc) as perfect_position
 from table
) temp
group by query
order by query, perfect_quality desc

select raw.query, search_quality/perfect_quality as final from raw
left join perfect
on raw.query = perfect.query

### Q2 假如某个query的所有relevance 都低于3，就定义为一个quality不好的search，找到这些query在所有query中占的比例。

In [253]:
df = table.copy()
df = df.groupby('query')['result_relevance'].max().reset_index()
df['bad_search'] = np.where(df.result_relevance<=3,1,0)
df.bad_search.sum()/(len(df)+0.0)

0.5

In [ ]:
select sum(bad_search)/count(*)
from
(select query, case when max(result_relevance) < 3 then 1 else 0 end as bad_search from table
group by query) temp



# Searching

In [254]:
search = pd.DataFrame({'Date':['2019-09-15', '2019-10-15','2019-09-15', '2019-10-15','2019-09-15', '2019-10-15'],
                      'Search_id':[1,2,3,4,5,6],
                      'User_id':[1,1,3,4,2,4],
                      'Age_group':[2,2,5,4,4,4],
                      'Search_query':['dog', 'dog', 'dog', 'cat', 'cat', 'cat']})

search_results = pd.DataFrame({'Date':['2019-09-15', '2019-10-15','2019-09-15', '2019-10-15','2019-09-15', '2019-10-15'],
                               'Search_id':[1,2,3,4,5,6],
                               'Result_id':[1,2,3,1,2,1],
                               'Result_type':['page', 'event', 'page', 'event', 'person', 'post'],
                               'Clicked':[1,0,1,1,1,0]})



### Q1: by each age group, how many unique users search for “dog” in last 7 days?


In [255]:
df = search.copy()
df = df[df.Search_query == 'dog']
# df = df[(datetime.datetime.now() - df.Date).dt.days <= 7]
df.groupby('Age_group')['User_id'].nunique()

Age_group
2    1
5    1
Name: User_id, dtype: int64

In [ ]:
select Age_group, count(distinct User_id) from search
where date_diff('day', Date, current_date)<=7 and Search_query = 'dog'
group by Age_group

In [ ]:
select Age_group, count(distinct User_id) from search
where Search_query = 'dog' and date_diff('day', Date, current_date) <=7
group by Age_group

### Q2: What are the top 10 search terms that are most likely to return at least one result about an event?


In [259]:
df = search_results.copy()
df = df[df.Result_type == 'event']
df = df.groupby('Search_id')['Result_id'].nunique().reset_index().rename(columns = {'Result_id':'event_cnt'})
df.merge(search, on = 'Search_id', how = 'left').sort_values(by = 'event_cnt', ascending = False)


,Search_id,event_cnt,Age_group,Date,Search_query,User_id
0,2,1,2,2019-10-15,dog,1


In [ ]:
select search_query, cnt
from
(select Search_id, count(*) as cnt from search_results
where Result_type = 'event'
group by Search_id) temp
left join search
on temp.Search_id = search.Search_id
order by cnt desc

In [ ]:
select Search_query
from 
(select Search_id from search_results
where Result_type = 'event'
group by Search_id
having count(Result_id)>1
order by count(Result_id) desc
limit 10) temp
left join 
search
on temp.Search_id = search.Search_id

# Friend

In [ ]:
friending = pd.DataFrame({'sender_id':[],
                          'receive_id':[],
                          'sent_date':[],
                          'accepted_date':[]
                         })
age_group = pd.DataFrame({'userid':[],
                         'age_group':[]})


### Q1 求过去四周，each day of week，friend request 的总数。一开始理解过了，算成28天每天，结果他说不对，我又说了一下理解， 发现是要按照MON/Tuest/Wed这样算

In [ ]:
select day_of_week(sent_date) as dayofweek, count(*)
from friending
where date_diff('day', sent_date, current_date) <=28
group by 1

In [ ]:
select day_of_week(sent_date) as dow, count(*) from friending
where date_diff('day', sent_date, current_date) <= 28
group by dow

### Q2 求过去一周，每个age group， average sent request。

In [ ]:
select age_group, count(*) from friending
left join age_group 
on friending.sender_id = age_group.userid
group by age_group

In [ ]:
select age_group, count(*) from friending
left join age_group
on friending.sender_id = age_group.userid
where date_diff('day', sent_date, current_date) <= 7
group by age_group


### Q3 acceptance rate in last week

In [ ]:
select count(accpeted_date)/count(sent_date) from friending
where date_diff('day', sent_date, current_date)<=7

In [ ]:
select sum(case when accepted_date is null then 0 else 1 end)/sum(case when sent_date is null then 0 else 1 end)
from friending
where date_diff('day', sent_date, current_date)<=7